# Tarea 1 Grupo

Aca explicas Personas importante exlicar sus atributos  y los metodos pasar dia y contacto

In [2]:
class Persona:
    def __init__(self,id,probabilidad_contagio):
        self._id = id
        self._probabilidad_contagio = probabilidad_contagio
        self.estado = "S" # S: sano; A asintomatico; E enfermo ; M muerto ; R recuperado
        self._dias_recuperacion = 0#cuenta regresiva a recuperacion
        self._dias_sintomas = 0 #cuenta regresiva a presentar sintomas
    @property
    def dias_recuperacion(self):
        return self._dias_recuperacion
    @dias_recuperacion.setter
    def dias_recuperacion(self, a):
        if(a <= 0):
            self._dias_recuperacion = 0
            self.estado = "R"
        self._dias_recuperacion = a
    @property
    def dias_sintomas(self):
        return self._dias_sintomas
    @dias_sintomas.setter
    def dias_sintomas(self, a):
        if(a <= 0):
            self._dias_sintomas = 0
            self.estado = "E"
        self._dias_sintomas = a
    def contacto(self,dado,dias_recuperacion,dias_sintomas):
        if self.estado == "S" and dado <= self._probabilidad_contagio:
            self.estado = "A"
            self._dias_recuperacion = dias_recuperacion
            self._dias_sintomas = min(dias_sintomas, dias_recuperacion)

    def pasar_dia (self):
        if self.estado in ["A","E"]:
            if self.estado == "A":
                self.dias_sintomas -= 1
            self.dias_recuperacion -= 1

    def __repr__(self):
        return self.estado+str(self._id)
    def __str__(self):
        if self.estado == "S":
            return f"Soy {self._id}, estoy Sano, P contagio {self._probabilidad_contagio}"
        if self.estado == "A":
            return f"Soy {self._id}, estoy Asintomatico,sintomas:{self._dias_sintomas}, recuperacion {self._dias_recuperacion}"
        if self.estado == "E":
            return f"Soy {self._id}, estoy Enfermo, recuperacion {self._dias_recuperacion}"
        if self.estado == "R":
            return f"Soy {self._id}, estoy Recuperado "

# Librerias 
explicacion de librerias

In [9]:
from collections import defaultdict
from personas import Persona
from simulation_parameters import simulation_parameters
from Codigo_Graficos import graficar_infectados
import os #https://stackoverflow.com/questions/7165749/open-file-in-a-relative-location-in-python
import re # https://stackoverflow.com/questions/1249388/removing-all-non-numeric-characters-from-string-in-python
import copy # copy deep copy
import time

explicacion

In [6]:
#archivo main.py
def crear_grafo ( personas, reuniones):
    """ Crea un default diccionario donde las llaves son los dias y los elemntos son una
    lista con los sets con los id de las personas que se juntaron """
    data_reuniones = defaultdict(list)
    with open (reuniones) as file:
        linea = file.__next__()
    inter = linea.split("({")

    for i in range(1,len(inter)):
        parcial = set(map(int,inter[i].split("}")[0].split(", ")))
        dia = int(re.sub("[^0-9]", "",inter[i].split("}")[1] ))
        data_reuniones[dia].append(parcial)
    #dia es el dia mas alto

    #cantidad de dia desde la infeccion que su estado de Enfermos con Sintomas
    """ Crea un diccionario donde se almacenan las persoans"""
    data_personas = dict()
    with open (personas) as file:
        linea = file.__next__().strip("{}")
    inter = linea.split(", ")
    for i in inter:
        persona = i.split(":")
        id = int(persona[0])
        data_personas[id]= Persona(id,float(persona[1]))
    return (data_personas,data_reuniones)


Aca explicas determinar contagiados importante notar la eficiencia de los sets 

In [7]:
#archivo main.py
def determinar_contagiados(grafo, p0 ,delta_dias):
    #gafo es in grafo formado por la estructura de la funcion crear_grafo
    #p0 es in integer con el indice del paciente 0
    #dia_final número de días transcurridos
    posibles = set()
    posibles.add(p0)
    data_reuniones = grafo[1]
    for t in range (0,delta_dias+1):
        if VERBOSO:
            print(f"POSIBLES al principio del DIA {t}",posibles)
        for reunion in data_reuniones[t]:
            if posibles.intersection(reunion):
                if VERBOSO:
                    print(F"Reunion infectada DIA{t}:",reunion, "culpables:",posibles.intersection(reunion))
                posibles = posibles.union(reunion)
    if VERBOSO:
        print(F"POSIBLES al final del dia {t}",posibles)
    return posibles

Aca puedes 

In [8]:
#Codigo_Graficos.py
import matplotlib.pyplot as plt
import numpy as np


def graficar_infectados(diccionario):
    print(diccionario)
    cantidad_casos_totales = diccionario["it"]
    cantidad_casos_recuperados = diccionario["r"]
    cantidad_casos_actuales = diccionario["i"]
    x = diccionario["t"]
    plt.plot(x, cantidad_casos_actuales,label= 'casos actuales')
    plt.plot(x, cantidad_casos_recuperados, label= 'casos recuperados')
    plt.plot(x, cantidad_casos_totales, label= 'casos totales')
    plt.ylabel('Cantidad de infectados')
    plt.xlabel('Días')
    plt.title("Infectados en función del tiempo")

    plt.legend(loc=0)
    plt.savefig("grafico.png")
    mostrar=plt.show()
    return mostrar